# TIME FOR THE TRANSFORMER!!!

Woohoo, I'm finally up to this point and I'm super excited. First some readings:

1. [3Blue1Brown](https://www.youtube.com/watch?v=wjZofJX0v4M&ab_channel=3Blue1Brown)
2. [Stat Quest](https://www.youtube.com/watch?v=zxQyTK8quyY&ab_channel=StatQuestwithJoshStarmer)
3. [The Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/)
4. [Attention is All you Need](https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf)
